In [1]:
import moviepy.editor as mpe
import numpy as np
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip, concatenate_videoclips, vfx

In [2]:
# Carrega o vídeo de entrada (certifique-se de que "video.mp4" existe no diretório)
video = VideoFileClip("video.mp4")

# Lista vazia para armazenar os clipes processados
clips = []

# Loop que divide o vídeo em segmentos de 20 segundos
for start in range(0, int(video.duration), 20):
    
    # Cria um subclipe do vídeo original, indo de 'start' até 'start+20' segundos
    # O 'min()' evita que ultrapasse o tempo total do vídeo
    clip = video.subclip(start, min(start + 20, video.duration))
    
    # Verifica se o índice do segmento é ímpar (para inverter a cada 20s)
    if (start // 20) % 2 == 1:
        
        # Aplica o efeito de espelhamento horizontal (inversão no eixo X)
        clip = clip.fx(vfx.mirror_x)
    
    # Adiciona o clipe (modificado ou não) à lista de clipes
    clips.append(clip)

# Concatena todos os clipes processados em um único vídeo
video_inverted = concatenate_videoclips(clips, method="compose")

# Exporta o vídeo final com:
# - codec de vídeo: libx264 (padrão para MP4)
# - codec de áudio: aac (padrão para áudio em MP4)
video_inverted.write_videofile("video_final.mp4", codec="libx264", audio_codec="aac")

Moviepy - Building video video_final.mp4.
MoviePy - Writing audio in video_finalTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video_final.mp4



Moviepy - Done !
Moviepy - video ready video_final.mp4


In [4]:
# Reduzir o som gradativamente
# Carrega o vídeo final processado anteriormente
video = VideoFileClip("video_final.mp4")

def volume_control(t):
    """Função que controla o volume em ciclos de 30 segundos:
       - Diminui linearmente de 1 (100%) a 0 (0%) nos primeiros 20s
       - Mantém em 0 (silêncio) nos últimos 10s de cada ciclo"""
    
    # Calcula a posição dentro do ciclo de 30s (0-29.999...)
    ciclo_pos = np.mod(t, 30)  # np.mod retorna o resto da divisão por 30
    
    # Cria um array de zeros com o mesmo formato que 't'
    volume = np.zeros_like(ciclo_pos, dtype=float)
    
    # Cria uma máscara para os tempos <= 20 segundos (parte com áudio)
    mask = ciclo_pos <= 20
    
    # Aplica redução linear: 1 → 0 em 20s (1 - t/20)
    volume[mask] = 1 - ciclo_pos[mask] / 20
    
    return volume  # Retorna os valores de volume para cada instante 't'

# Ajusta o volume do áudio (processamento estéreo)
audio_adjusted = video.audio.fl(lambda gf, t: 
    np.column_stack([
        # Canal esquerdo: multiplica o sinal original pelo volume controlado
        gf(t)[:, 0] * volume_control(t),  
        
        # Canal direito: mesma operação para o segundo canal
        gf(t)[:, 1] * volume_control(t)   
    ])
)

# Atualiza o vídeo com o áudio modificado
video = video.set_audio(audio_adjusted)

# Salva o vídeo com as configurações otimizadas:
video.write_videofile(
    "video_volume_ciclico.mp4",  # Nome do arquivo de saída
    codec="libx264",             # Codec de vídeo (padrão para MP4)
    audio_codec="aac",           # Codec de áudio (padrão para MP4)
    fps=video.fps,               # Mantém o FPS original
    threads= 4                   # Usa 4 threads para codificação mais rápida
)

t:  43%|████▎     | 2607/6057 [00:54<00:56, 61.31it/s, now=None]

Moviepy - Building video video_volume_ciclico.mp4.
MoviePy - Writing audio in video_volume_ciclicoTEMP_MPY_wvf_snd.mp4


t:  43%|████▎     | 2607/6057 [01:00<00:56, 61.31it/s, now=None]

MoviePy - Done.
Moviepy - Writing video video_volume_ciclico.mp4



t:  43%|████▎     | 2607/6057 [02:58<00:56, 61.31it/s, now=None]

Moviepy - Done !
Moviepy - video ready video_volume_ciclico.mp4


In [5]:
# Cortar o trecho entre 60 e 80 segundos
# ------------------------------------------------------------
# Carrega o vídeo com o volume já ajustado ("video_volume_ciclico.mp4")
# Certifique-se de que este arquivo existe no diretório atual
video = VideoFileClip("video_volume_ciclico.mp4")

# Corta um trecho específico do vídeo:
# - Início: 60 segundos (1 minuto)
# - Fim: 80 segundos (1 minuto e 20 segundos)
# O método subclip cria um novo clip com apenas essa porção
trecho_60_80 = video.subclip(60, 80)

# Salva o trecho cortado em um novo arquivo MP4:
# - "trecho_60_80.mp4": nome do arquivo de saída
# - codec="libx264": codec de vídeo de alta qualidade
# - audio_codec="aac": codec de áudio padrão para MP4
# - fps=video.fps: mantém a mesma taxa de quadros do vídeo original
trecho_60_80.write_videofile(
    "trecho_60_80.mp4",
    codec="libx264",
    audio_codec="aac",
    fps=video.fps
)

t:  43%|████▎     | 2607/6057 [03:24<00:56, 61.31it/s, now=None]

Moviepy - Building video trecho_60_80.mp4.
MoviePy - Writing audio in trecho_60_80TEMP_MPY_wvf_snd.mp4


t:  43%|████▎     | 2607/6057 [03:25<00:56, 61.31it/s, now=None]

MoviePy - Done.
Moviepy - Writing video trecho_60_80.mp4



t:  43%|████▎     | 2607/6057 [03:34<00:56, 61.31it/s, now=None]

Moviepy - Done !
Moviepy - video ready trecho_60_80.mp4


In [6]:
# Adicionar o trecho cortado ao final do vídeo principal
# Carrega o vídeo principal
# ATENÇÃO: O arquivo "vide.mp4" deve existir na pasta do projeto
video_principal = VideoFileClip("video_volume_ciclico.mp4")

# Carrega o trecho que será adicionado no final
# ATENÇÃO: O arquivo "tre.mp4" deve existir na pasta do projeto
trecho = VideoFileClip("trecho_60_80.mp4")

# Concatena o vídeo principal com o trecho no final
# - O método "compose" garante que os vídeos com diferentes resoluções sejam compatíveis
# - Os vídeos serão unidos na ordem da lista [primeiro o principal, depois o trecho]
video_final = concatenate_videoclips(
    [video_principal, trecho],  # Lista dos vídeos a serem concatenados
    method="compose"            # Método de composição padrão
)

# Salva o vídeo final com as seguintes configurações:
# - Nome do arquivo: "video_com_trecho_no_final.mp4"
# - Codec de vídeo: libx264 (padrão para arquivos MP4)
# - Codec de áudio: aac (padrão para qualidade de áudio)
# - Mantém o mesmo FPS (quadros por segundo) do vídeo principal
video_final.write_videofile(
    "video_com_trecho_no_final.mp4",
    codec="libx264",            # Codec de vídeo de alta qualidade
    audio_codec="aac",          # Codec de áudio padrão
    fps=video_principal.fps     # Mantém a fluidez do vídeo original
)

t:  43%|████▎     | 2607/6057 [08:16<00:56, 61.31it/s, now=None]

Moviepy - Building video video_com_trecho_no_final.mp4.
MoviePy - Writing audio in video_com_trecho_no_finalTEMP_MPY_wvf_snd.mp4


t:  43%|████▎     | 2607/6057 [08:21<00:56, 61.31it/s, now=None]

MoviePy - Done.
Moviepy - Writing video video_com_trecho_no_final.mp4



t:  43%|████▎     | 2607/6057 [11:34<00:56, 61.31it/s, now=None]

Moviepy - Done !
Moviepy - video ready video_com_trecho_no_final.mp4
